In [12]:
with open(r"D:\testData.txt", "r") as reader:
    txtFile= reader.read()
#print(txtFile)

In [21]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

sentences =sent_tokenize(txtFile) # NLTK function
total_documents = len(sentences)
print(total_documents)

52


In [34]:
# 2.Create the Frequency matrix of the words in each sentence.

from nltk.stem import PorterStemmer


def build_frequency_matrix(sentences):
    frequency_matrix = {}
    stopWords = set(nltk.corpus.stopwords.words("english"))
    ps = PorterStemmer()

    for sent in sentences:
        freq_table = {}
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            word = ps.stem(word)
            if word in stopWords:
                continue

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[sent[:15]] = freq_table

    return frequency_matrix

In [36]:
# Function Call
frequency_matrix=build_frequency_matrix(sentences)
#frequency_matrix

In [37]:
# 3. Calculate TermFrequency and generate a matrix
def build_tf_matrix(freq_matrix):
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence

        tf_matrix[sent] = tf_table

    return tf_matrix

In [41]:
# Function call
tf_matrix=build_tf_matrix(frequency_matrix)
#tf_matrix

In [47]:
# 4. Creating a table for documents per words

def build_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table

In [50]:
# function call
word_per_doc_table=build_documents_per_words(frequency_matrix)
#word_per_doc_table

In [51]:
# 5. Calculate IDF and generate a matrix

def build_idf_matrix(freq_matrix, count_doc_per_words, total_documents):
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix

In [54]:
# function call
import math
idf_matrix=build_idf_matrix(frequency_matrix, word_per_doc_table, total_documents)
#idf_matrix

In [55]:
# 6. Calculate TF-IDF and generate a matrix

def build_tf_idf_matrix(tf_matrix, idf_matrix):
    tf_idf_matrix = {}

    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(),
                                                    f_table2.items()):  # here, keys are the same in both the table
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[sent1] = tf_idf_table

    return tf_idf_matrix

In [57]:
# function call
tf_idf_matrix = build_tf_idf_matrix(tf_matrix, idf_matrix)
#tf_idf_matrix

In [59]:
# 7. Score the sentences

def calculate_score_sentences(tf_idf_matrix) -> dict:
    
    sentenceValue = {}

    for sent, f_table in tf_idf_matrix.items():
        total_score_per_sentence = 0

        count_words_in_sentence = len(f_table)
        for word, score in f_table.items():
            total_score_per_sentence += score

        sentenceValue[sent] = total_score_per_sentence / count_words_in_sentence

    return sentenceValue

In [61]:
# function call
sentenceValue =calculate_score_sentences(tf_idf_matrix)
# sentenceValue

In [62]:
# 8. Find the threshold

def calculate_average_score(sentenceValue) -> int:
   
    sumValues = 0
    for entry in sentenceValue:
        sumValues += sentenceValue[entry]

    # Average value of a sentence from original summary_text
    average = (sumValues / len(sentenceValue))

    return average

In [63]:
# function call
average = calculate_average_score(sentenceValue)
average

0.15647933364110622

In [64]:
# 9. Generate the summary

def generate_summary(sentences, sentenceValue, threshold):
    sentence_count = 0
    summary = ''

    for sentence in sentences:
        if sentence[:15] in sentenceValue and sentenceValue[sentence[:15]] >= (threshold):
            summary += " " + sentence
            sentence_count += 1

    return summary

In [66]:
# function call
summary= generate_summary(sentences, sentenceValue, average)
summary

' Have you experienced this before? To be honest, I don’t have the answers. Same failure, yet different responses. Who is right and who is wrong? Neither. Each person has a different mindset that decides their outcome. It was at that point their biggest breakthrough came. Perhaps all those years of perseverance finally paid off. If you settle for less, you will receive less than you deserve and convince yourself you are justified to receive it. It must come from within you. Gnaw away at your problems until you solve them or find a solution. Where are you settling in your life right now? Could you be you playing for bigger stakes than you are? Vision + desire + dedication + patience + daily action leads to astonishing success. Success is a fickle and long game with highs and lows. So become intentional on what you want out of life. Commit to it. Nurture your dreams. Don’t leave your dreams to chance.'